In [1]:
import os
import pathlib
import enum
import math

import cv2
import matplotlib.pyplot as plt
import mediapipe as mp
import numpy as np
import torch
from PIL import Image
from torchvision import models, transforms
from torchvision.models import ResNet50_Weights

%matplotlib inline

objc[7851]: Class CaptureDelegate is implemented in both /Users/sangyun/miniconda3/envs/deeplearning/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x11d452490) and /Users/sangyun/miniconda3/envs/deeplearning/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x129608860). One of the two will be used. Which one is undefined.
objc[7851]: Class CVWindow is implemented in both /Users/sangyun/miniconda3/envs/deeplearning/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x11d4524e0) and /Users/sangyun/miniconda3/envs/deeplearning/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x11fe4ca68). One of the two will be used. Which one is undefined.
objc[7851]: Class CVView is implemented in both /Users/sangyun/miniconda3/envs/deeplearning/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x11d452508) and /Users/sangyun/miniconda3/envs/deeplearning/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x11fe4ca90). One of the tw

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

Configure Model Path

In [3]:
DIR = os.path.dirname(os.path.abspath("__file__"))

save_file_name = os.path.join(DIR, 'models/resnet50-cuda-v2.pt')
checkpoint_path = os.path.join(DIR, 'models/resnet50-cuda-v2.pth')

Configure CPU / GPU Inference Usage

In [4]:
load_on_gpu = False # Set True to load on GPU
device = torch.device("cpu")

if load_on_gpu:
  if torch.cuda.is_available():
    device = torch.device("cuda")
  elif torch.backends.mps.is_available():
    device = torch.device("mps")

print(f'Device: {device}')

Device: cpu


In [5]:
class PoseLandmark(enum.IntEnum):
  """The 33 pose landmarks."""
  NOSE = 0
  LEFT_EYE_INNER = 1
  LEFT_EYE = 2
  LEFT_EYE_OUTER = 3
  RIGHT_EYE_INNER = 4
  RIGHT_EYE = 5
  RIGHT_EYE_OUTER = 6
  LEFT_EAR = 7
  RIGHT_EAR = 8
  MOUTH_LEFT = 9
  MOUTH_RIGHT = 10
  LEFT_SHOULDER = 11
  RIGHT_SHOULDER = 12
  LEFT_ELBOW = 13
  RIGHT_ELBOW = 14
  LEFT_WRIST = 15
  RIGHT_WRIST = 16
  LEFT_PINKY = 17
  RIGHT_PINKY = 18
  LEFT_INDEX = 19
  RIGHT_INDEX = 20
  LEFT_THUMB = 21
  RIGHT_THUMB = 22
  LEFT_HIP = 23
  RIGHT_HIP = 24
  LEFT_KNEE = 25
  RIGHT_KNEE = 26
  LEFT_ANKLE = 27
  RIGHT_ANKLE = 28
  LEFT_HEEL = 29
  RIGHT_HEEL = 30
  LEFT_FOOT_INDEX = 31
  RIGHT_FOOT_INDEX = 32

In [6]:
_POSE_LANDMARKS = frozenset([
    PoseLandmark.LEFT_SHOULDER, PoseLandmark.RIGHT_SHOULDER,
    PoseLandmark.LEFT_ELBOW, PoseLandmark.RIGHT_ELBOW,
    PoseLandmark.LEFT_WRIST, PoseLandmark.RIGHT_WRIST,
    PoseLandmark.LEFT_HIP, PoseLandmark.RIGHT_HIP,
    PoseLandmark.LEFT_KNEE, PoseLandmark.RIGHT_KNEE,
    PoseLandmark.LEFT_ANKLE, PoseLandmark.RIGHT_ANKLE
])

In [7]:
POSE_CONNECTIONS = frozenset([(0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5),
                              (5, 6), (6, 8), (9, 10), (11, 12), (11, 13),
                              (13, 15), (15, 17), (15, 19), (15, 21), (17, 19),
                              (12, 14), (14, 16), (16, 18), (16, 20), (16, 22),
                              (18, 20), (11, 23), (12, 24), (23, 24), (23, 25),
                              (24, 26), (25, 27), (26, 28), (27, 29), (28, 30),
                              (29, 31), (30, 32), (27, 31), (28, 32)])


print(POSE_CONNECTIONS)

frozenset({(15, 21), (16, 20), (18, 20), (3, 7), (14, 16), (23, 25), (28, 30), (11, 23), (27, 31), (6, 8), (15, 17), (24, 26), (16, 22), (4, 5), (5, 6), (29, 31), (12, 24), (23, 24), (0, 1), (9, 10), (1, 2), (0, 4), (11, 13), (30, 32), (28, 32), (15, 19), (16, 18), (25, 27), (26, 28), (12, 14), (17, 19), (2, 3), (11, 12), (27, 29), (13, 15)})


In [8]:
def load_checkpoint(path):
    """Load a PyTorch model checkpoint

    Params
    --------
        path (str): saved model checkpoint. Must start with `model_name-` and end in '.pth'

    Returns
    --------
        None, save the `model` to `path`

    """
    # Load in checkpoint
    checkpoint = torch.load(path, map_location=device)

    model = models.resnet50(weights=ResNet50_Weights.DEFAULT)
    # Make sure to set parameters as not trainable
    for param in model.parameters():
        param.requires_grad = False
    model.fc = checkpoint['fc']

    # Load in the state dict
    model.load_state_dict(torch.load(save_file_name, map_location=device))

    total_params = sum(p.numel() for p in model.parameters())
    print(f'{total_params:,} total parameters.')
    total_trainable_params = sum(
        p.numel() for p in model.parameters() if p.requires_grad)
    print(f'{total_trainable_params:,} total gradient parameters.')

    print()

    if load_on_gpu:
        model = model.to(device)

    # Model basics
    model.class_to_idx = checkpoint['class_to_idx']
    model.idx_to_class = checkpoint['idx_to_class']
    model.epochs = checkpoint['epochs']

    # Optimizer
    optimizer = checkpoint['optimizer']
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    return model, optimizer

In [9]:
model, optimizer = load_checkpoint(path=checkpoint_path)

24,033,861 total parameters.
525,829 total gradient parameters.



In [10]:
from typing import List, Mapping, Tuple, Optional, Union
import dataclasses

In [11]:
_VISIBILITY_THRESHOLD = 0.5
_BGR_CHANNELS = 3

WHITE_COLOR = (224, 224, 224)
BLACK_COLOR = (0, 0, 0)
RED_COLOR = (0, 0, 255)
GREEN_COLOR = (0, 128, 0)
BLUE_COLOR = (255, 0, 0)

In [12]:
@dataclasses.dataclass
class DrawingSpec:
  # Color for drawing the annotation. Default to the white color.
  color: Tuple[int, int, int] = WHITE_COLOR
  # Thickness for drawing the annotation. Default to 2 pixels.
  thickness: int = 2
  # Circle radius. Default to 2 pixels.
  circle_radius: int = 2

In [13]:
def _normalized_to_pixel_coordinates(
    normalized_x: float, normalized_y: float, image_width: int,
    image_height: int) -> Union[None, Tuple[int, int]]:
  """Converts normalized value pair to pixel coordinates."""

  # Checks if the float value is between 0 and 1.
  def is_valid_normalized_value(value: float) -> bool:
    return (value > 0 or math.isclose(0, value)) and (value < 1 or
                                                      math.isclose(1, value))

  if not (is_valid_normalized_value(normalized_x) and
          is_valid_normalized_value(normalized_y)):
    # TODO: Draw coordinates even if it's outside of the image bounds.
    return (None, None)
  x_px = min(math.floor(normalized_x * image_width), image_width - 1)
  y_px = min(math.floor(normalized_y * image_height), image_height - 1)
  return x_px, y_px

In [14]:
class Landmark_coord(object):
    def __init__(self,name):
        self.name = name
        self.coor = [ (None, None) for i in range(30) ]
        self.visibility = []

In [15]:
Landmark_arr = []
for landmark in PoseLandmark:
    Landmark_arr.append(Landmark_coord(landmark))
print(Landmark_arr[1].name)
print(Landmark_arr[1].coor)
print(Landmark_arr[1].visibility)

def returnObj(name):
    for landmark in Landmark_arr:
        if landmark.name == name:
            return landmark


PoseLandmark.LEFT_EYE_INNER
[(None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None)]
[]


In [16]:
MHI_DURATION = 30

COLOR = []
RADIUS = []
for idx in range(1,MHI_DURATION+1):
  ratio = (idx-1)/(MHI_DURATION-1)
  result = 5*ratio
  result += 3
  RADIUS.append(int(result))
RADIUS.reverse()

MHI_DURATION_FIRST = 15
for result_idx in range(1,MHI_DURATION_FIRST+1):
    ratio = (result_idx-1)/(MHI_DURATION_FIRST-1)
    COLOR.append((0,int(255*ratio), int(255*(1-ratio))))

MHI_DURATION_SECOND = 15
for result_idx in range(1,MHI_DURATION_SECOND+1):
    ratio = (result_idx-1)/(MHI_DURATION_SECOND-1)
    COLOR.append((int(255*ratio), int(255*(1-ratio)), 0))

In [17]:
def draw(image, landmark_list, connections):
  if not landmark_list:
    return
  if image.shape[2] != _BGR_CHANNELS:
    raise ValueError('Input image must contain three channel bgr data.')

  image_rows, image_cols, _ = image.shape
  cv2.rectangle(image, (0,0), (image_cols,image_rows), (0,0,0), cv2.FILLED)

  for idx, landmark in enumerate(_POSE_LANDMARKS):
    print('landmark: ', landmark)
    print('landmark name: ', Landmark_arr[landmark].name)
    print('landmark coor: ', Landmark_arr[landmark].coor)

    Landmark_arr[landmark].coor.pop()
    if landmark_list[landmark].visibility < _VISIBILITY_THRESHOLD:
      Landmark_arr[landmark].coor.insert(0,(None, None))
    else:
      landmark_px = _normalized_to_pixel_coordinates(landmark_list[idx].x, landmark_list[idx].y,
                                                    image_cols, image_rows)
      Landmark_arr[landmark].coor.insert(0,landmark_px)
  #print("Landmark: ", Landmark_arr[1].name, " ", Landmark_arr[1].coor)

  '''
  if connections:
    # Draws the connections if the start and end landmarks are both visible.
    for connection in connections:
      start_idx = connection[0]
      end_idx = connection[1]

      if start_idx in _POSE_LANDMARKS and end_idx in _POSE_LANDMARKS:
        #print('start idx: ', start_idx, 'end idx: ', end_idx)
        #print(start_idx in _POSE_LANDMARKS)
        #print(end_idx in _POSE_LANDMARKS)
        #print(Landmark_arr[start_idx].name)
        x_coor = Landmark_arr[start_idx].coor[0]
        y_coor = Landmark_arr[end_idx].coor[0]
        #print(x_coor, y_coor)
        if (x_coor != (None, None) and y_coor != (None, None)):
          #print(x_coor, y_coor)
          cv2.line(image, x_coor,
                  y_coor, DrawingSpec.color,
                  DrawingSpec.thickness)

  return image
  '''
  
  # Draws landmark points after finishing the connection lines, which is
  # aesthetically better.
  for landmark_px in Landmark_arr:
    # Fill color into the circle
    #print(landmark_px)
    for idx in range(MHI_DURATION):
      cv2.circle(image, landmark_px.coor[idx], RADIUS[idx],
                  COLOR[idx], cv2.FILLED , cv2.LINE_AA) 


  return image

In [19]:
with torch.inference_mode():
  cap = cv2.VideoCapture(0)
  with mp_pose.Pose(
      min_detection_confidence=0.5,
      min_tracking_confidence=0.5) as pose:
      
    while cap.isOpened():
      success, frame = cap.read()
      if not success:
        print("Ignoring empty camera frame.")
        continue

      # To improve performance, optionally mark the frame as not writeable to
      # pass by reference.
      frame.flags.writeable = False
      frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      results = pose.process(frame)

      # Draw the pose annotation on the frame.
      frame.flags.writeable = True
      frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

      if results.pose_landmarks:
        landmark_pose = results.pose_landmarks.landmark

        image = draw(frame, landmark_pose, POSE_CONNECTIONS)

        '''
        draw_landmarks(
            frame,
            results,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        '''
        # Flip the frame horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
        if (cv2.waitKey(10) & 0xFF == ord('q')):
          break
      
  cap.release()
  cv2.destroyAllWindows()
  cv2.waitKey(1)
  cv2.waitKey(1)
  cv2.waitKey(1)




landmark:  PoseLandmark.LEFT_SHOULDER
landmark name:  PoseLandmark.LEFT_SHOULDER
landmark coor:  [(1041, 494), (1040, 496), (1041, 498), (1041, 501), (1048, 511), (1048, 517), (1048, 519), (1048, 519), (1048, 521), (1046, 520), (1046, 502), (1050, 480), (1047, 479), (1040, 494), (1039, 510), (1036, 517), (1033, 517), (1025, 522), (1023, 519), (1023, 518), (1023, 517), (1028, 510), (1028, 510), (1037, 495), (1039, 494), (1039, 509), (1042, 520), (1045, 524), (1040, 529), (1040, 551)]
landmark:  PoseLandmark.RIGHT_SHOULDER
landmark name:  PoseLandmark.RIGHT_SHOULDER
landmark coor:  [(1075, 444), (1074, 444), (1075, 446), (1075, 447), (1079, 449), (1081, 449), (1081, 449), (1082, 448), (1082, 448), (1082, 446), (1082, 437), (1084, 429), (1082, 430), (1077, 440), (1076, 449), (1072, 452), (1068, 452), (1063, 455), (1061, 452), (1064, 450), (1065, 448), (1070, 442), (1070, 441), (1075, 434), (1077, 435), (1077, 444), (1085, 454), (1089, 460), (1089, 467), (1091, 486)]
landmark:  PoseLandmar

KeyboardInterrupt: 

: 